In [28]:
from pathlib import Path
import os

import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics
from sklearn.pipeline import Pipeline
from pygam import LinearGAM, s, f, te, l
from sklearn import ensemble

from data_processing import *
# hacky shit
# from data_processing.data_processing import *

In [3]:
MAIN_PATH = Path(os.getcwd()).parent

## Load data

In [4]:
org_df = pd.read_csv(os.path.join(MAIN_PATH, "data", "data.csv"))

# copied from load() function but modified to aggregate data and not drop it

org_df['czas'] = pd.to_datetime(org_df['czas'], utc=True)
org_df.set_index('czas', inplace=True)


temp_zuz = pd.read_csv(os.path.join(MAIN_PATH, "data", "temp_zuz_fixed.csv"))


temp_zuz['Czas'] = pd.to_datetime(temp_zuz['Czas'], utc=True)
temp_zuz.set_index('Czas', inplace=True)
temp_zuz = temp_zuz.rename(columns={'Czas': 'czas'})

org_df.head(3)

,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001fir01307.daca.pv,001fir01308.daca.pv,001fir01309.daca.pv,001fir01310.daca.pv,001fir01311.daca.pv,001fir01312.daca.pv,...,001uxm0rf01.daca.pv,001uxm0rf02.daca.pv,001uxm0rf03.daca.pv,037tix00254.daca.pv,037tix00264.daca.pv,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s
czas,,,,,,,,,,,,,,,,,,,,,
2020-09-30 22:00:00+00:00,56.729077,54.724422,11.966905,22.493207,108.702362,112.280388,110.013796,108.761583,110.604901,104.500427,...,92.090454,92.024078,92.405281,24.665309,24.526161,4.48,8.98,8.6,4.55,9.87
2020-09-30 22:01:00+00:00,54.771942,54.734675,12.026410,22.593412,108.846612,112.334600,109.906645,108.879083,110.644046,104.371977,...,92.090534,92.024206,92.405239,24.663982,24.526161,4.48,8.98,8.6,4.55,9.87
2020-09-30 22:02:00+00:00,54.695816,54.774163,11.953671,22.428933,108.827640,112.362508,109.970932,108.844765,110.621665,104.387912,...,92.090613,92.024334,92.405196,24.662656,24.526161,4.48,8.98,8.6,4.55,9.87


## Simple features based on correlation

In [5]:
df_cleared = pd.DataFrame(index=org_df.index)

In [6]:
# srednia po 'TEMP POD 2 WARSTWĄ WYMURÓWKI [°C]'
TIX1 = org_df.columns[org_df.columns.str.contains('001tix')].values
df_cleared['TIX1'] = org_df[TIX1].mean(axis='columns')

In [7]:
# srednia po 'WODY df KOLEKTORÓW [°C]'
TIR = org_df.columns[org_df.columns.str.contains('tir')].values
df_cleared['TIR'] = org_df[TIR].mean(axis='columns')

In [8]:
# dodanie reg nadawy koncentratu
FCX = org_df.columns[org_df.columns.str.contains('fcx')].values
df_cleared[FCX] = org_df[FCX]

In [9]:
# sumaryczna moc cieplna
NIR = org_df.columns[org_df.columns.str.contains('nir')].values
df_cleared[NIR] = org_df[NIR]

In [10]:
df_cleared.shape

(540492, 7)

### Aggregation and lagging

In [11]:
# Renaming columns

feature_desc_df = pd.read_csv(os.path.join(MAIN_PATH, "data_processing", "feature_desc.csv"), index_col='name')

def feature_desc(name):
        try:
            if type(name)==str:
                return feature_desc_df.loc[name]['desc']
            else:
                return feature_desc_df.loc[name]['desc'].values
        except:
            return name


col_names = list(df_cleared.columns.values)

for col_name in col_names:
    df_cleared = df_cleared.rename(columns={col_name: feature_desc(col_name)})

In [12]:
df_cleared.columns

Index(['TIX1', 'TIR', 'REG NADAWY KONCENTRATU LIW1 [Mg/h]',
       'REG NADAWY KONCENTRATU LIW2 [Mg/h]',
       'REG KONCENTRAT PRAZONY LIW3 [Mg/h]', 'REG PYL ZWROT LIW4 [Mg/h]',
       'SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW]'],
      dtype='object')

# TUTAJ PARAMTER

In [13]:
# Aggregating
template_df = df_cleared.copy()
AGGREGATE_INTERVAL = 20

agg_df = aggregate(df_cleared, AGGREGATE_INTERVAL)
agg_df.head(3)

/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW]
czas,,,,,,,
2020-09-30 22:00:00+00:00,418.986435,29.084745,55.089606,54.911403,12.001296,22.534135,14.400833
2020-09-30 22:20:00+00:00,418.970708,29.041929,54.823559,54.965885,12.001552,22.433780,14.621633
2020-09-30 22:40:00+00:00,418.942088,28.985106,54.857146,55.005352,12.002782,22.491250,14.831200


In [14]:
agg_df.shape

(27026, 7)

In [15]:
# Adding lagged featues

lagged_df = agg_df.copy()

for i in range(60 // AGGREGATE_INTERVAL):
    lagged_df = pd.concat([lagged_df, agg_df.shift(i+1).add_prefix(f'{i+1}th_agg_')], axis=1)

# delete rows with nans created by shifts
lagged_df = lagged_df.iloc[(60 // AGGREGATE_INTERVAL):, :]
    
lagged_df.head(3)

,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],1th_agg_TIX1,1th_agg_TIR,1th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],...,2th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],2th_agg_REG PYL ZWROT LIW4 [Mg/h],2th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],3th_agg_TIX1,3th_agg_TIR,3th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],3th_agg_REG NADAWY KONCENTRATU LIW2 [Mg/h],3th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],3th_agg_REG PYL ZWROT LIW4 [Mg/h],3th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW]
czas,,,,,,,,,,,,,,,,,,,,,
2020-09-30 23:00:00+00:00,418.911518,28.995099,54.860078,54.951873,11.517584,22.558489,14.977075,418.942088,28.985106,54.857146,...,12.001552,22.433780,14.621633,418.986435,29.084745,55.089606,54.911403,12.001296,22.534135,14.400833
2020-09-30 23:20:00+00:00,418.880421,28.964434,54.739989,55.002890,10.989900,22.507531,15.071495,418.911518,28.995099,54.860078,...,12.002782,22.491250,14.831200,418.970708,29.041929,54.823559,54.965885,12.001552,22.433780,14.621633
2020-09-30 23:40:00+00:00,418.849777,28.953018,54.753259,54.980088,11.003144,21.428431,15.114872,418.880421,28.964434,54.739989,...,11.517584,22.558489,14.977075,418.942088,28.985106,54.857146,55.005352,12.002782,22.491250,14.831200


In [16]:
lagged_df.shape

(27023, 28)

In [17]:
lagged_df.head(5)

,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],1th_agg_TIX1,1th_agg_TIR,1th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],...,2th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],2th_agg_REG PYL ZWROT LIW4 [Mg/h],2th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],3th_agg_TIX1,3th_agg_TIR,3th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],3th_agg_REG NADAWY KONCENTRATU LIW2 [Mg/h],3th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],3th_agg_REG PYL ZWROT LIW4 [Mg/h],3th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW]
czas,,,,,,,,,,,,,,,,,,,,,
2020-09-30 23:00:00+00:00,418.911518,28.995099,54.860078,54.951873,11.517584,22.558489,14.977075,418.942088,28.985106,54.857146,...,12.001552,22.433780,14.621633,418.986435,29.084745,55.089606,54.911403,12.001296,22.534135,14.400833
2020-09-30 23:20:00+00:00,418.880421,28.964434,54.739989,55.002890,10.989900,22.507531,15.071495,418.911518,28.995099,54.860078,...,12.002782,22.491250,14.831200,418.970708,29.041929,54.823559,54.965885,12.001552,22.433780,14.621633
2020-09-30 23:40:00+00:00,418.849777,28.953018,54.753259,54.980088,11.003144,21.428431,15.114872,418.880421,28.964434,54.739989,...,11.517584,22.558489,14.977075,418.942088,28.985106,54.857146,55.005352,12.002782,22.491250,14.831200
2020-10-01 00:00:00+00:00,418.820072,29.010398,54.794222,54.978824,10.990859,19.988716,15.187157,418.849777,28.953018,54.753259,...,10.989900,22.507531,15.071495,418.911518,28.995099,54.860078,54.951873,11.517584,22.558489,14.977075
2020-10-01 00:20:00+00:00,418.791159,29.041578,54.830487,55.001058,11.009614,19.980993,15.006475,418.820072,29.010398,54.794222,...,11.003144,21.428431,15.114872,418.880421,28.964434,54.739989,55.002890,10.989900,22.507531,15.071495


In [18]:
lagged_df = lagged_df.join(temp_zuz['temp_zuz'])
lagged_df = lagged_df.dropna(axis=0)

# dodanie temperatury żużla
lagged_df = lagged_df.rename(columns={'temp_zuz': 'TEMP_ZUZ'})

In [19]:
lagged_df.head(5)

,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],1th_agg_TIX1,1th_agg_TIR,1th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],...,2th_agg_REG PYL ZWROT LIW4 [Mg/h],2th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],3th_agg_TIX1,3th_agg_TIR,3th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],3th_agg_REG NADAWY KONCENTRATU LIW2 [Mg/h],3th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],3th_agg_REG PYL ZWROT LIW4 [Mg/h],3th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],TEMP_ZUZ
czas,,,,,,,,,,,,,,,,,,,,,
2020-09-30 23:00:00+00:00,418.911518,28.995099,54.860078,54.951873,11.517584,22.558489,14.977075,418.942088,28.985106,54.857146,...,22.433780,14.621633,418.986435,29.084745,55.089606,54.911403,12.001296,22.534135,14.400833,1295.0
2020-10-01 00:00:00+00:00,418.820072,29.010398,54.794222,54.978824,10.990859,19.988716,15.187157,418.849777,28.953018,54.753259,...,22.507531,15.071495,418.911518,28.995099,54.860078,54.951873,11.517584,22.558489,14.977075,1303.0
2020-10-01 01:00:00+00:00,418.735368,29.083472,54.805588,54.975587,10.991591,21.010127,14.617195,418.763390,29.063903,54.907207,...,19.980993,15.006475,418.820072,29.010398,54.794222,54.978824,10.990859,19.988716,15.187157,1303.0
2020-10-01 02:00:00+00:00,418.649147,29.080083,54.898549,54.956940,13.006610,20.919631,14.240552,418.678843,29.086580,54.896649,...,20.794801,14.583456,418.735368,29.083472,54.805588,54.975587,10.991591,21.010127,14.617195,1306.0
2020-10-01 03:00:00+00:00,418.568565,28.935882,54.924863,54.987031,13.001828,23.019041,13.721479,418.593275,28.973077,54.783087,...,22.845271,14.029273,418.649147,29.080083,54.898549,54.956940,13.006610,20.919631,14.240552,1303.0


In [20]:
# shift last temp
lagged_df = pd.concat([lagged_df, lagged_df[['TEMP_ZUZ']].shift(1).add_prefix('TWOJA_STARA_')], axis=1)

lagged_df = lagged_df.iloc[1:, :]

In [21]:
df = lagged_df.copy()

In [22]:
df.head(25)

,TIX1,TIR,REG NADAWY KONCENTRATU LIW1 [Mg/h],REG NADAWY KONCENTRATU LIW2 [Mg/h],REG KONCENTRAT PRAZONY LIW3 [Mg/h],REG PYL ZWROT LIW4 [Mg/h],SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],1th_agg_TIX1,1th_agg_TIR,1th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],...,2th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],3th_agg_TIX1,3th_agg_TIR,3th_agg_REG NADAWY KONCENTRATU LIW1 [Mg/h],3th_agg_REG NADAWY KONCENTRATU LIW2 [Mg/h],3th_agg_REG KONCENTRAT PRAZONY LIW3 [Mg/h],3th_agg_REG PYL ZWROT LIW4 [Mg/h],3th_agg_SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW],TEMP_ZUZ,TWOJA_STARA_TEMP_ZUZ
czas,,,,,,,,,,,,,,,,,,,,,
2020-10-01 00:00:00+00:00,418.820072,29.010398,54.794222,54.978824,10.990859,19.988716,15.187157,418.849777,28.953018,54.753259,...,15.071495,418.911518,28.995099,54.860078,54.951873,11.517584,22.558489,14.977075,1303.0,1295.0
2020-10-01 01:00:00+00:00,418.735368,29.083472,54.805588,54.975587,10.991591,21.010127,14.617195,418.763390,29.063903,54.907207,...,15.006475,418.820072,29.010398,54.794222,54.978824,10.990859,19.988716,15.187157,1303.0,1303.0
2020-10-01 02:00:00+00:00,418.649147,29.080083,54.898549,54.956940,13.006610,20.919631,14.240552,418.678843,29.086580,54.896649,...,14.583456,418.735368,29.083472,54.805588,54.975587,10.991591,21.010127,14.617195,1306.0,1303.0
2020-10-01 03:00:00+00:00,418.568565,28.935882,54.924863,54.987031,13.001828,23.019041,13.721479,418.593275,28.973077,54.783087,...,14.029273,418.649147,29.080083,54.898549,54.956940,13.006610,20.919631,14.240552,1303.0,1306.0
2020-10-01 04:00:00+00:00,418.496671,28.889696,54.918190,54.954007,13.001099,22.998772,13.830014,418.519754,28.864799,54.863837,...,13.645798,418.568565,28.935882,54.924863,54.987031,13.001828,23.019041,13.721479,1304.0,1303.0
2020-10-01 05:00:00+00:00,418.446714,28.998518,54.826771,55.003252,13.012226,23.680318,14.520465,418.460660,28.879882,54.771071,...,14.014987,418.496671,28.889696,54.918190,54.954007,13.001099,22.998772,13.830014,1302.0,1304.0
2020-10-01 06:00:00+00:00,418.405892,29.136704,54.753442,55.029215,13.009340,23.975382,15.053035,418.419221,29.076030,54.913940,...,14.540211,418.446714,28.998518,54.826771,55.003252,13.012226,23.680318,14.520465,1303.0,1302.0
2020-10-01 07:00:00+00:00,418.408846,29.073628,54.904902,54.951986,13.009900,24.021465,14.923139,418.400834,29.063547,54.890699,...,14.811232,418.405892,29.136704,54.753442,55.029215,13.009340,23.975382,15.053035,1297.0,1303.0
2020-10-01 08:00:00+00:00,418.435215,29.044314,55.131060,55.283339,12.542630,23.802410,14.905969,418.425558,29.056845,54.806638,...,14.771825,418.408846,29.073628,54.904902,54.951986,13.009900,24.021465,14.923139,1293.0,1297.0


## Split data

In [23]:
train, val, test = split(df)

In [24]:
X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

In [25]:
ridge_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                        ('ridge', linear_model.Ridge(alpha=1, solver='cholesky'))])
ridge_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model.predict(X_val))

21.79438111808414

## Most of the code above is ignored and moved to generate_dataset function that takes interval as paramter

# Testing different models

In [26]:
def generate_dataset(arg_df, interval, temp_zuz):
    
    internal_df = aggregate(arg_df, interval)

    internal_final_df = internal_df.copy(deep=True)
    for i in range(60 // interval):
        internal_final_df = pd.concat([internal_final_df, internal_df.shift(i+1).add_prefix(f'{i+1}th_agg_')], axis=1)

    # delete rows with nans created by shifts
    internal_final_df = internal_final_df.iloc[(60 // interval):, :]


    internal_final_df = internal_final_df.join(temp_zuz['temp_zuz'])
    internal_final_df = internal_final_df.dropna(axis=0)
    # dodanie temperatury żużla
    internal_final_df = internal_final_df.rename(columns={'temp_zuz': 'TEMP_ZUZ'})
    
    internal_final_df = pd.concat([internal_final_df, internal_final_df[['TEMP_ZUZ']].shift(1).add_prefix('TWOJA_STARA_')], axis=1)

    internal_final_df = internal_final_df.iloc[1:, :]

    return internal_final_df

## Ridge regression

In [249]:
solvers = ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
alphas = [.1, .5, 1]

for interval in [5, 10, 15, 20, 30]:
    # creating dataset
    df = generate_dataset(template_df, interval, temp_zuz)
    train, val, test = split(df)
    X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
    y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]
    print(f'------------------ INTERVAL: {interval}min --------------------')
    for solver in solvers:
        for alpha in alphas:
            ridge_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                                    ('ridge', linear_model.Ridge(alpha=alpha, solver=solver))])
            ridge_model.fit(X_train, y_train)
            mse = metrics.mean_squared_error(y_val, ridge_model.predict(X_val))
            print(interval, solver, alpha, mse)

/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


------------------ INTERVAL: 5min --------------------
5 svd 0.1 22.07113709376668
5 svd 0.5 22.019815367167283
5 svd 1 21.972066687391276
5 cholesky 0.1 22.071137093766524
5 cholesky 0.5 22.019815367167084
5 cholesky 1 21.972066687391152
5 lsqr 0.1 21.581615164090444
5 lsqr 0.5 21.578286471979496
5 lsqr 1 21.57426290855367
5 sparse_cg 0.1 22.093917611459233
5 sparse_cg 0.5 22.01356792682001
5 sparse_cg 1 21.921882882991014
5 sag 0.1 21.80112239054707
5 sag 0.5 21.786446632419803
5 sag 1 21.767556208022718
5 saga 0.1 21.631188623473
5 saga 0.5 21.620970217431996
5 saga 1 21.614766895126632
------------------ INTERVAL: 10min --------------------
10 svd 0.1 21.063739999076855
10 svd 0.5 21.03109392281493
10 svd 1 20.992574454555943


/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


10 cholesky 0.1 21.063739999077345
10 cholesky 0.5 21.031093922815078
10 cholesky 1 20.992574454555953
10 lsqr 0.1 20.670451450192417
10 lsqr 0.5 20.665860614261106
10 lsqr 1 20.660503159486105
10 sparse_cg 0.1 21.042935150627923
10 sparse_cg 0.5 21.009341749979217
10 sparse_cg 1 20.97348878527448
10 sag 0.1 20.964177914601443
10 sag 0.5 20.940293345370215
10 sag 1 20.913618224713662
10 saga 0.1 20.886309168573625
10 saga 0.5 20.869677824350276
10 saga 1 20.85130654211868
------------------ INTERVAL: 15min --------------------
15 svd 0.1 21.185661974556233
15 svd 0.5 21.147256801326886
15 svd 1 21.104795633986214
15 cholesky 0.1 21.185661974556485
15 cholesky 0.5 21.147256801326986
15

/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


 cholesky 1 21.10479563398628
15 lsqr 0.1 20.71433537842927
15 lsqr 0.5 20.715925261427223
15 lsqr 1 20.717925274648923
15 sparse_cg 0.1 21.13056468445879
15 sparse_cg 0.5 21.130913801222928
15 sparse_cg 1 21.112290706728402
15 sag 0.1 21.102410565491773
15 sag 0.5 21.074203680301093
15 sag 1 21.04334127250895
15 saga 0.1 21.03887137767338
15 saga 0.5 21.01796400870533
15 saga 1 20.99527604540053
------------------ INTERVAL: 20min --------------------
20 svd 0.1 21.86738974387616
20 svd 0.5 21.833029266104123
20 svd 1 21.7943811180842
20 cholesky 0.1 21.867389743876025
20 cholesky 0.5 21.833029266104013
20 cholesky 1 21.79438111808414
20 lsqr 0.1 21.54152976612529


/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


20 lsqr 0.5 21.541564846483993
20 lsqr 1 21.541628209019926
20 sparse_cg 0.1 21.900348396686855
20 sparse_cg 0.5 21.859588440963893
20 sparse_cg 1 21.817168665103427
20 sag 0.1 21.78705420849821
20 sag 0.5 21.76155451760332
20 sag 1 21.734813895894842
20 saga 0.1 21.72452588800611
20 saga 0.5 21.707868141668037
20 saga 1 21.689538429538878
------------------ INTERVAL: 30min --------------------
30 svd 0.1 21.48011567949126
30 svd 0.5 21.441880639773114
30 svd 1 21.402742232651082
30 cholesky 0.1 21.480115679490893
30 cholesky 0.5 21.441880639772865
30 cholesky 1 21.402742232650844
30 lsqr 0.1 21.142297367050354


/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


30 lsqr 0.5 21.14429386372647
30 lsqr 1 21.146798929891457
30 sparse_cg 0.1 21.48137289500443
30 sparse_cg 0.5 21.441782190099495
30 sparse_cg 1 21.401957155430715
30 sag 0.1 21.410586618241357
30 sag 0.5 21.382911131171575
30 sag 1 21.354960761506046
30 saga 0.1 21.355566548948648
30 saga 0.5 21.33635101141895
30 saga 1 21.31698886926159


## Ridge Regression + Polynomial Features

In [ ]:
solvers = ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
alphas = [.1, .5, 1]
degrees = [2]

for interval in [5, 10, 15, 20, 30]:
    # creating dataset
    df = generate_dataset(template_df, interval, temp_zuz)
    train, val, test = split(df)
    X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
    y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]
    print(f'------------------ INTERVAL: {interval}min --------------------')
    for solver in solvers:
        for alpha in alphas:
            for degree in degrees:
                ridge_model = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=degree)),
                                        ('normalization', preprocessing.StandardScaler()),
                                        ('ridge', linear_model.Ridge(alpha=alpha, solver=solver))])
                ridge_model.fit(X_train, y_train)
                mse = metrics.mean_squared_error(y_val, ridge_model.predict(X_val))
                print(interval, solver, alpha, degree, mse)

## SGD Regression

In [267]:
penalties = ['l1', 'l2', 'elasticnet']

for interval in [5, 10, 15, 20, 30]:
    # creating dataset
    df = generate_dataset(template_df, interval, temp_zuz)
    train, val, test = split(df)
    X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
    y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]
    print(f'------------------ INTERVAL: {interval}min --------------------')
    for penalty in penalties:
        sgd_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                          ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty=penalty, learning_rate='adaptive'))])
        sgd_model.fit(X_train, y_train)
        mse = metrics.mean_squared_error(y_val, sgd_model.predict(X_val))
        print(interval, penalty, mse)

/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


------------------ INTERVAL: 5min --------------------
5 l1 4.40736721055518e+19
5 l2 2.9385228005225005e+19
5 elasticnet 1.8953456424774033e+19


/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


------------------ INTERVAL: 10min --------------------
10 l1 21.08800951356584
10 l2 21.484399680004206
10 elasticnet 22.707295571703337
------------------ INTERVAL: 15min --------------------


/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


15 l1 21.00304260520398
15 l2 21.786946429760036
15 elasticnet 21.075828265283384
------------------ INTERVAL: 20min --------------------


/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


20 l1 21.61959936708876
20 l2 21.43380864407839
20 elasticnet 21.637296553823877
------------------ INTERVAL: 30min --------------------
30 l1 21.17992026970367


/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


30 l2 21.167767304079582
30 elasticnet 21.218378057770977


## Random Forest

In [ ]:
for interval in [5, 10, 15, 20, 30]:
    # creating dataset
    df = generate_dataset(template_df, interval, temp_zuz)
    train, val, test = split(df)
    X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
    y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]
    print(f'------------------ INTERVAL: {interval}min --------------------')
    for i in range(1, 10):
        rfg_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                                ('forest', ensemble.RandomForestRegressor(max_depth=i, random_state=42))])
        rfg_model = rfg_model.fit(X_train, y_train)
        mse = metrics.mean_squared_error(y_val, rfg_model.predict(X_val))
        print(i, mse)

/home/przemek/projects/hackathons/cuvalley/linear_regression/data_processing.py:127: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if not type(agg_df['czas']) is pd.datetime:


------------------ INTERVAL: 5min --------------------
1 36.11224139011172
2 28.453702245351213
3 26.369678428115435
4 26.35249918814626
5 29.12156435396585
6 27.99988720578228


# Lagged but without aggregation
Lagged features from N minutes before measuremnet of temperature

In [150]:
temp_zuz['temp_zuz']

count    11384.000000
mean      1303.389318
std          7.531903
min       1190.000000
25%       1300.000000
50%       1304.000000
75%       1308.000000
max       1338.000000
Name: temp_zuz, dtype: float64

In [264]:
MINUTES_BEFORE = 50

df_lagged_no_agg = template_df.copy()

df_lagged_no_agg.reset_index(inplace=True)
mask = (df_lagged_no_agg['czas'].dt.minute == 0) | (df_lagged_no_agg['czas'].dt.minute > 60 - MINUTES_BEFORE)

df_lagged_no_agg = df_lagged_no_agg.loc[mask]
df_lagged_no_agg.set_index('czas', inplace=True)
                   
final_df = df_lagged_no_agg.copy(deep=True)
for i in range(MINUTES_BEFORE):
    final_df = pd.concat([final_df, df_lagged_no_agg.shift(MINUTES_BEFORE-i).add_prefix(f'{MINUTES_BEFORE-i}mins_ago_')], axis=1)
    

# delete rows with nans created by shifts
final_df = final_df.iloc[MINUTES_BEFORE:, :]


final_df = final_df.join(temp_zuz['temp_zuz'])

final_df = final_df.dropna(axis=0)
# dodanie temperatury żużla
final_df = final_df.rename(columns={'temp_zuz': 'TEMP_ZUZ'})

final_df = pd.concat([final_df, final_df[['TEMP_ZUZ']].shift(1).add_prefix('TWOJA_STARA_')], axis=1)

final_df = final_df.iloc[1:, :]


In [265]:
train, val, test = split(final_df)

X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

ridge_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                        ('ridge', linear_model.Ridge(alpha=1, solver='cholesky'))])
ridge_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model.predict(X_val))

23.11784462677727